# Starting to build control
 Sends commands to a remote device on a serial port.  If an unknown commmand sends the #
whole lot.  Assuming a REPL loop rather like a terminal.
Version 1 after crash.

### To align (Servo version)

- Switch on if not on
- Run the code if not already running
- Use the command `td` to test the DVD.  This will position the tool head just above the DVD drive.
- Put a dummy CD in
- Turn the power off
- Rotate the arm by hand to be exactly over the DVD
- Turn the power on
- Restart and clear output on code (the COM port will have gone and should now be back)

## DVD door commands

In [ ]:
import string
from ctypes import windll

class DVDError(Exception):
    pass

def get_drives():
    drives = []
    bitmask = windll.kernel32.GetLogicalDrives()
    print(bitmask)
    for letter in 'ABCDEFGHIJKLMNOPQ':
        if bitmask & 1:
            drives.append(letter)
        bitmask >>= 1

    return drives

def drive_is_d():
    drive_list = get_drives()
    assert len(drive_list) == 2
    assert drive_list[1] == 'D'
    return True

def open_dvd():
    windll.WINMM.mciSendStringW(u"set cdaudio door open", None, 0, None)

def close_dvd():
    windll.WINMM.mciSendStringW(u"set cdaudio door closed", None, 0, None)

if drive_is_d():
    print('Found D drive all ok')
else:
    raise DVDError("Didn't find D drive")

## Commands to control robot

In [ ]:
import time
import serial

# configure the serial connections (the parameters differs on the device you are connecting to)
ser = serial.Serial(
    port='COM6',
    baudrate=115200
)

def robot(command):
        ser.write((command + '\r\n').encode('utf-8'))
        out = ''
        # let's wait two seconds before reading output (let's give device time to answer)
        time.sleep(2)  # TODO make waiting more intelligent, ie wait only if have to.
        while ser.inWaiting() > 0:
            out += ser.read(1).decode('utf-8')
        if out != '':
            print("Robot>" + out)

ser.isOpen()

print('Enter your commands below.\r\nType "e" to leave the application.')
robot('import control1')  # Tends to do a soft reset at end of prog

def toolhead_safe():
    """Put to park and shutdown"""
    robot('control1.th.park()')  # Make toolhead safe
    time.sleep(2)  # TODO Poll status
    robot('control1.th.shutdown()')  # Make toolhead safe

def raise_toolhead():
    """Raise the toolhead and wait for it to be finished"""
    robot('control1.z_axis.nudge(44, run_time=90)')  # Move to top
    time.sleep(2)  # TODO Poll status
    robot('control1.z_axis.shutdown()')

def make_safe():
    """Put the robot into a safe position to start or end a series of moves"""
    toolhead_safe()
    raise_toolhead()
    
class Rotator:
    def __init__(self):
        self.old_pos = -1

    def rotate_head_anti_hystersis(self, new_position):
        if new_position != self.old_pos:
            pre_move = max(0, new_position - 5)
            robot(f'control1.rotate.wait_set_position({pre_move})')
            time.sleep(1)  # Needs to wait a little longer to settle
            robot(f'control1.rotate.wait_set_position({new_position})')
            self.old_pos = new_position  # Store the old poistion

    def in_bin(self):
        self.rotate_head_anti_hystersis(86)

    def dvd(self):
        self.rotate_head_anti_hystersis(49)

    def out_bin(self):
        self.rotate_head_anti_hystersis(14)

rotate = Rotator()
    
def pickup_from_dvd_drive():
    open_dvd()
    rotate.dvd()
    robot('control1.th.release()')
    robot('control1.z_axis.nudge(49, run_time=90)')  # Move to surface
    robot('control1.th.grip()')
    time.sleep(1)  # Let it be securely gripped?
    raise_toolhead()

def pickup_from_in_bin():
    rotate.in_bin()
    robot('control1.th.release()')
    robot('control1.z_axis.nudge(49, run_time=120)')  # Deeper than DVD so move further
    robot('control1.th.grip()')
    time.sleep(1)  # Let it be securely gripped?
    raise_toolhead()

def drop_on_dvd_drive():
    open_dvd()
    rotate.dvd()
    robot('control1.z_axis.nudge(49, run_time=40)')  # Move nearer dvd
    time.sleep(4)  # TODO poll
    robot('control1.th.release()')
    time.sleep(1)  # Let the release happen
    robot('control1.th.park()')
    time.sleep(2)  # Let the park happen before shutdown
    robot('control1.th.shutdown()')

def drop_on_out_bin():
    rotate.out_bin()
    robot('control1.z_axis.nudge(49, run_time=20)')  # Move nearer bin
    time.sleep(2)  # TODO poll
    robot('control1.th.release()')
    time.sleep(1)  # Let the release happen
    robot('control1.th.park()')
    time.sleep(2)  # Let the park happen before shutdown
    robot('control1.th.shutdown()')

while 1:
    # get keyboard input
    user_input = input("->")
    if user_input == 'e':
        print('Shutting serial down')
        ser.close()
        print('Breaking loop to stop')
        break
    elif user_input == 'l':  # Load disk into drive
        make_safe()
        pickup_from_in_bin()
        drop_on_dvd_drive()
        make_safe()
        close_dvd()
    elif user_input == 'u':  # Unload disk from drive
        make_safe()
        pickup_from_dvd_drive()
        close_dvd()
        drop_on_out_bin()
        make_safe()
        close_dvd()
    elif user_input == 'td':  # Test DVD alignment
        make_safe()
        open_dvd()
        rotate.dvd()
        robot('control1.z_axis.nudge(49, run_time=30)')  # Move nearer dvd
    elif user_input == 'td1':  # Test Pickup DVD from drive and drop it back
        make_safe()
        pickup_from_dvd_drive()
        drop_on_dvd_drive()
        make_safe()
        close_dvd()
    elif user_input == 'tl':  # Load disk into drive
        make_safe()
        rotate.in_bin()
        robot('control1.z_axis.nudge(49, run_time=30)')  # Move nearer dvd
    elif user_input == 'tu':  # Unload disk from drive
        make_safe()
        rotate.out_bin()
        robot('control1.z_axis.nudge(49, run_time=30)')  # Move nearer dvd
    elif user_input == 'n':
        robot('control1.th.park()')
    elif user_input == 'm':
        robot('control1.th.grip()')
    elif user_input == ',':
        robot('control1.th.release()')
    elif user_input == '.':
        robot('control1.th.shutdown()')
    elif len(user_input) > 2 and user_input[0] == 'r':
        position = float(user_input[1:])
        print(f'New position = {position}')
        robot(f'control1.rotate.wait_set_position({position})')
    elif user_input == 's':
        robot('control1.status()')
    elif user_input == 's10':
        for i in range(10):
            robot('control1.status()')
    else:
        robot(user_input)


Enter your commands below.
Type "e" to leave the application.
Robot>import control1
>>> 
->l
Robot>control1.th.park()
>>> 
Robot>control1.th.shutdown()
>>> 
Robot>control1.z_axis.nudge(44, run_time=90)
Shutdown Z Axis nudge
>>> 
Robot>control1.z_axis.shutdown()
>>> 
Robot>control1.rotate.wait_set_position(81)
>>> 
Robot>control1.rotate.wait_set_position(86)
>>> 
Robot>control1.th.release()
>>> 
Robot>control1.z_axis.nudge(49, run_time=120)

Robot>Shutdown Z Axis nudge
>>> control1.th.grip()
>>> control1.z_axis.nudge(44, run_time=90)

Robot>Shutdown Z Axis nudge
>>> control1.z_axis.shutdown()
>>> control1.rotate.wait_set_position(44)

Robot>>>> control1.rotate.wait_set_position(49)
>>> control1.z_axis.nudge(49, run_time=40)
Shutdown Z Axis nudge
>>> control1.th.release()
>>> 
Robot>control1.th.park()
>>> 
Robot>control1.th.shutdown()
>>> 
Robot>control1.th.park()
>>> 
Robot>control1.th.shutdown()
>>> 
Robot>control1.z_axis.nudge(44, run_time=90)

Robot>Shutdown Z Axis nudge
>>> control1